# ATP Report

This report is ana anlysis of the Active Transportation Program Application Cycles. Currently, this analysis is grouped at the **cycle** level. 

In [1]:
import warnings
warnings.filterwarnings('ignore')


In [2]:
%%capture
import intake
import numpy as np
import pandas as pd
import geopandas as gpd
from calitp import to_snakecase
from dla_utils import _dla_utils
from IPython.display import HTML, Markdown
from siuba import *
from shared_utils import geography_utils

import altair as alt

import _data_cleaning
import _report_utils

In [3]:
import fiona

from ipyleaflet import Map, GeoJSON, projections, basemaps, GeoData, LayersControl, WidgetControl, GeoJSON, LegendControl
from ipywidgets import Text, HTML

In [4]:
GCS_FILE_PATH = 'gs://calitp-analytics-data/data-analyses/dla/atp/'


In [5]:
## read in data

In [6]:
df = _report_utils.read_in_joined_data()

In [7]:
df = (_report_utils.reorder_namecol(df,
                    og_name_col= 'a1_imp_agcy_name',
                    new_name_col= 'imp_agency_name_new', 
                    split_on = ", ",
                   order_on ='pt2_pt1'))

In [8]:
pd.set_option("display.max_columns",500)

In [9]:
## paramaters cell -- not using 
#district = 4

In [10]:
## parameters cell
cycle = 5

In [11]:
## subset df to just the district
#df = df_all>>filter(_.a2_ct_dist==district)

In [12]:
df = df>>filter(_.project_cycle==cycle)

In [13]:
display(HTML("<h2>Quick Stats</h2>"))

display(HTML(f"Out of {len(df)} Active Transportation Program Project Applications, "
            f"there are <strong>{(df>>filter(_.awarded=='Y')).project_app_id.nunique()} "
            f"projects</strong> that recieved funding over "
            f"{df.project_cycle.nunique()} cycles"))


HTML(value='<h2>Quick Stats</h2>')

HTML(value='Out of 450 Active Transportation Program Project Applications, there are <strong>49 projects</stro…

In [14]:
display(HTML(f"<h4> What were the application outcomes "
             # f"for District {district} "
             f"in Cycle {cycle}?</h4>"))
display(HTML(_dla_utils.pretify_tables((df>>count(_.data_origin)))))

HTML(value='<h4> What were the application outcomes in Cycle 5?</h4>')

HTML(value='<style type="text/css">\n#T_0a10b_ th {\n  text-align: center;\n}\n#T_0a10b_row0_col0, #T_0a10b_ro…

In [15]:
#df>>group_by(_.awarded)>>count(_.a2_county)>>arrange(_.a2_county)

## Mapping

In [16]:
import folium

In [17]:
df_map = _report_utils.prep_mapping_data(df)

In [18]:
## from amanda's notebook for Project prioritization: https://github.com/cal-itp/data-analyses/blob/main/project_prioritization/_utils.py
shapes = "https://gis.data.ca.gov/datasets/0144574f750f4ccc88749004aca6eb0c_0.geojson?outSR=%7B%22latestWkid%22%3A3857%2C%22wkid%22%3A102100%7D"
district_shapes = gpd.read_file(f"{shapes}").to_crs(epsg=4326)  

by_dist = _report_utils.nunique_by_geography(df_map,
                         district_shapes,
                         groupby_cols = ['a2_ct_dist', 'data_origin'],
                         agg_col=  'project_app_id',
                         geodf_merge_on_col = 'DISTRICT',
                         sum_merge_on_col = 'a2_ct_dist')

In [19]:
display(HTML("<h3>Number of Applications by County</h3>"))

HTML(value='<h3>Number of Applications by County</h3>')

In [52]:
# m = by_dist.explore(
#      column="Application", 
#      legend=True,
#     cmap = 'Oranges',
#      legend_kwds=dict(colorbar=True),
#      tooltip=["DISTRICT", "Application"], 
#      name="Number of Applications by County")

# df_map.explore(
#      m=m, 
#      color="data_origin", 
#      marker_kwds=dict(radius=3, fill=True),
#      tooltip=["data_origin", "project_cycle", "imp_agency_name_new", "a1_imp_agcy_city", "a2_info_proj_name"], 
#      tooltip_kwds=dict(labels=True),
#      name="Project Locations"
# )

# folium.TileLayer('Stamen Toner', control=True).add_to(m) 
# folium.LayerControl().add_to(m)

# m

In [21]:
districts = _report_utils.get_district_level_data(by_dist, df_map)

In [22]:
display(HTML("<h3>Mapping Option for Assembly Districts (or others)</h3>"))

HTML(value='<h3>Mapping Option for Assembly Districts (or others)</h3>')

In [53]:
# _report_utils.map_districts(districts, 
#                  district_type= "Assembly District",
#                  district_type_col= "District Type",
#                  mapping_col= "Success Rate",
#               tool_tip_cols= ['District Type', 'District', 'Success Rate'])

In [24]:
display(HTML("<h3>Mapping Option for All Districts (dropdown)</h3>"))

HTML(value='<h3>Mapping Option for All Districts (dropdown)</h3>')

In [25]:
#_report_utils.add_dropdown_map(districts)

In [26]:
## Metrics

In [27]:
# unique_agencies = df>>group_by(_.project_cycle, _.a2_county, _.data_origin)>>summarize(n_unique_agency= _.imp_agency_name_new.nunique())

In [28]:
# unique_agencies =(spread(unique_agencies, "data_origin", "n_unique_agency"))

In [29]:
# unique_agencies = unique_agencies.rename(columns={"a2_county":"county_name"})

In [30]:
# unique_agencies['Application'].fillna(0, inplace=True)
# unique_agencies['Funded'].fillna(0, inplace=True)

In [31]:
# unique_agencies['Application'] = unique_agencies['Application'].astype('int32')
# unique_agencies['Funded'] = unique_agencies['Funded'].astype('int32')

In [32]:
# display(HTML("<h3>Number of Unique Agencies By County</h3>"))
# display(HTML(_dla_utils.pretify_tables(unique_agencies)))


## Application and Funded Project Details by District

In [33]:
from shared_utils import altair_utils

In [34]:
funded = df>>filter(_.awarded=='Y')

### District 1 - Eureka

In [35]:
_report_utils.map_dist_proj(funded, df, 1)

alt.Chart(...)

Implemeting Agency Name,Project Name,County,Total Atp Funds,Total Project Cost
City of Clearlake,Dam Road Extension & South Center Drive Bike/Pedestrian Improvements,Lake,"$ 997,000.00","$ 997,000.00"
City of Arcata,Arcata Annie & Mary Trail Connectivity Project,Humboldt,"$ 4,220,000.00","$ 5,286,000.00"


Implemeting Agency Name,Total Applications,Success Rate
City of Arcata,1,100.00%
City of Clearlake,1,100.00%
City of Eureka,2,0.00%
City of Rio Dell,1,0.00%
Humboldt County,1,0.00%
Humboldt County Association of Governments,1,0.00%
Lake County,1,0.00%
Mendocino Council of Governments,1,0.00%
Round Valley Indians Tribe,1,0.00%


### District 2 - Redding

In [36]:
_report_utils.map_dist_proj(funded, df, 2)

alt.Chart(...)

Implemeting Agency Name,Project Name,County,Total Atp Funds,Total Project Cost
Karuk Tribe,Happy Camp Complete Streets Project,Siskiyou,"$ 9,971,000.00","$ 12,221,000.00"
City of Redding,Turtle Bay to Downtown Gap Completion Project,Shasta,"$ 2,665,000.00","$ 3,935,000.00"
Shasta County,Cottonwood Active Transportation Trunk Line Express (CATTLE) Network,Shasta,"$ 7,056,000.00","$ 17,844,000.00"
Siskiyou County Transportation Commission,Siskiyou- Regional Active Transportation Plan,Siskiyou,"$ 202,000.00","$ 212,000.00"
City of Redding,Victor Ave & Cypress Ave Active Transportation (VCAT) Project,Shasta,"$ 7,822,000.00","$ 10,409,000.00"


Implemeting Agency Name,Total Applications,Success Rate
City of Redding,3,66.67%
Karuk Tribe,1,100.00%
Shasta County,3,33.33%
Siskiyou County Transportation Commission,1,100.00%
City of Shasta Lake,1,0.00%
Department of Transportation,1,0.00%
Modoc County,1,0.00%
Shasta County RTPA,1,0.00%


### District 3 - Marysville

In [37]:
_report_utils.map_dist_proj(funded, df, 3)

alt.Chart(...)

Implemeting Agency Name,Project Name,County,Total Atp Funds,Total Project Cost
El Dorado County,Pollock Pines - Pony Express Trail Bicycle and Pedestrian Improvements,El Dorado,"$ 1,440,000.00","$ 2,000,000.00"
Sacramento County,South Sacramento County Safe Routes to School Project,Sacramento,"$ 1,946,000.00","$ 1,946,000.00"
City of Sacramento,Franklin Boulevard Complete Street Project,Sacramento,"$ 9,323,000.00","$ 16,265,000.00"
City of West Sacramento,Sycamore Trail (Phase 2) Bicycle/Pedestrian Overcrossing,Yolo,"$ 3,500,000.00","$ 11,538,000.00"


Implemeting Agency Name,Total Applications,Success Rate
City of Sacramento,1,100.00%
City of West Sacramento,2,50.00%
El Dorado County,8,12.50%
Sacramento County,3,33.33%
5002,1,0.00%
5447,1,0.00%
Butte County,1,0.00%
City of Biggs,1,0.00%
City of Citrus Heights,1,0.00%
City of Colusa,1,0.00%


In [50]:
#df>>filter(_.a1_imp_agcy_name=="5002")

In [51]:
#df>>filter(_.a1_imp_agcy_name=="5447")

### District 4 - Oakland 

In [38]:
_report_utils.map_dist_proj(funded, df, 4)

alt.Chart(...)

Implemeting Agency Name,Project Name,County,Total Atp Funds,Total Project Cost
City of Oakland,7th Street Connection Project,Alameda,"$ 14,180,000.00","$ 21,037,000.00"
Santa Clara County,Active and Safe Routes to a Healthier City,Santa Clara,"$ 2,510,000.00","$ 2,510,000.00"
City of Oakland,East Oakland Neighborhood Bike Routes,Alameda,"$ 17,269,000.00","$ 21,859,000.00"
Contra Costa County,North Bailey Road Active Transportation Corridor,Contra Costa,"$ 6,159,000.00","$ 6,845,000.00"
City of Fairfield,West Texas Street Complete Streets Project,Solano,"$ 10,903,000.00","$ 16,922,000.00"


Implemeting Agency Name,Total Applications,Success Rate
City of Oakland,6,33.33%
City of Fairfield,1,100.00%
Contra Costa County,6,16.67%
Santa Clara County,1,100.00%
Alameda County,6,0.00%
Alameda County Transportation Commission,1,0.00%
Bay Area Toll Authority,1,0.00%
California Department of Transportation,1,0.00%
City of Berkeley,2,0.00%
City of Concord,1,0.00%


### District 5 - San Luis Obispo

In [39]:
_report_utils.map_dist_proj(funded, df, 5)

alt.Chart(...)

Implemeting Agency Name,Project Name,County,Total Atp Funds,Total Project Cost
City of Santa Cruz,Santa Cruz Rail Trail Segment 7 Phase 2 Construction,Santa Cruz,"$ 9,184,000.00","$ 12,030,000.00"
City of Seaside,Broadway Ave Complete Street Corridor,Monterey,"$ 12,041,000.00","$ 14,001,000.00"
City of Watsonville,Safer Access to Pajaro Valley High School and Beyond,Santa Cruz,"$ 11,709,000.00","$ 15,823,000.00"
City of Santa Barbara,Upper De La Vina Street Gap Closure and Safe Crossings,Santa Barbara,"$ 1,998,000.00","$ 1,998,000.00"
San Luis Obispo County,San Luis Obispo County-Bob Jones Trail Gap Closure,San Luis Obispo,"$ 18,248,000.00","$ 23,414,000.00"


Implemeting Agency Name,Total Applications,Success Rate
City of Santa Barbara,4,25.00%
City of Santa Cruz,3,33.33%
City of Seaside,1,100.00%
City of Watsonville,1,100.00%
San Luis Obispo County,5,20.00%
California Department of Transportation,1,0.00%
City of Atascadero,1,0.00%
City of Buellton,1,0.00%
City of Carpinteria,1,0.00%
City of El Paso De Robles,1,0.00%


### District 6 - Frenso

In [40]:
_report_utils.map_dist_proj(funded, df, 6)

alt.Chart(...)

Implemeting Agency Name,Project Name,County,Total Atp Funds,Total Project Cost
City of Corcoran,Corcoran Safe Routes to School,Kings,"$ 1,998,000.00","$ 1,998,000.00"
Fresno County,Biola Community Sidewalks,Fresno,"$ 1,255,000.00","$ 1,498,000.00"
City of Delano,ATP-5 SRTS Intersection Enhancement and NI Work Plan,Kern,"$ 1,164,000.00","$ 1,178,000.00"
City of Fresno,Kids Crossing: Safe Routes to School in South Fresno,Fresno,"$ 1,636,000.00","$ 1,636,000.00"
City of Porterville,Butterfield Stage Corridor (W North Grand Ave to College Ave),Tulare,"$ 7,100,000.00","$ 7,750,000.00"
City of Huron,City of Huron Bicyclist and Pedestrian Safety Improvement Project,Fresno,"$ 1,769,000.00","$ 1,969,000.00"


Implemeting Agency Name,Total Applications,Success Rate
City of Corcoran,1,100.00%
City of Delano,2,50.00%
City of Fresno,3,33.33%
City of Huron,1,100.00%
City of Porterville,4,25.00%
Fresno County,1,100.00%
5216,1,0.00%
California Department of Transportation,1,0.00%
City of Bakersfield,5,0.00%
City of Clovis,1,0.00%


### District 7 - Los Angeles

In [41]:
_report_utils.map_dist_proj(funded, df, 7)

alt.Chart(...)

Implemeting Agency Name,Project Name,County,Total Atp Funds,Total Project Cost
City of Bell Gardens,Bell Gardens Complete Streets Improvements - Phase 1,Los Angeles,"$ 6,499,000.00","$ 6,999,000.00"
City of Maywood,City of Maywood Active Transportation Plan,Los Angeles,"$ 263,000.00","$ 263,000.00"
City of Santa Monica,Stewart-Pennsylvania Safety Enhancement Project,Los Angeles,"$ 3,196,000.00","$ 4,000,000.00"
City of Long Beach,Pacific Avenue Cycle Track,Los Angeles,"$ 7,498,000.00","$ 8,332,000.00"
City of South El Monte,South El Monte Safe Routes to School Pedestrian Safety Project,Los Angeles,"$ 1,637,000.00","$ 1,637,000.00"
City of Los Angeles,Connecting Canoga Park Through Safety and Urban Cooling Improvements,Los Angeles,"$ 30,731,000.00","$ 38,655,000.00"
City of Los Angeles,"SRTS Carver Middle, Ascot Avenue and Harmony Elementary Schools Project",Los Angeles,"$ 6,030,000.00","$ 6,700,000.00"
City of Los Angeles,SRTS Berendo Middle and 3 Feeder Elementary Schools Safety Project,Los Angeles,"$ 9,951,000.00","$ 11,057,000.00"
City of Los Angeles,SRTS Panorama City Elementary School Project,Los Angeles,"$ 6,149,000.00","$ 6,832,000.00"
City of Long Beach,Downtown Long Beach Walkable Corners,Los Angeles,"$ 7,893,000.00","$ 8,771,000.00"


Implemeting Agency Name,Total Applications,Success Rate
City of Los Angeles,12,33.33%
City of Long Beach,4,50.00%
City of Bell Gardens,1,100.00%
City of Maywood,1,100.00%
City of Santa Monica,2,50.00%
City of South El Monte,1,100.00%
Ventura County,3,33.33%
California Department of Transportation,1,0.00%
City of Alhambra,1,0.00%
City of Avalon,1,0.00%


### District 8 - San Bernardino

In [42]:
_report_utils.map_dist_proj(funded, df, 8)

alt.Chart(...)

Implemeting Agency Name,Project Name,County,Total Atp Funds,Total Project Cost
San Bernardino County,Muscoy Area Safe Routes to School Pedestrian Improvements Project,San Bernardino,"$ 1,881,000.00","$ 2,355,000.00"
Riverside County,Riverside County Safe Routes for All - San Jacinto,Riverside,"$ 600,000.00","$ 600,000.00"
City of Perris,City of Perris Bike and Pedestrian Network Project,Riverside,"$ 1,931,000.00","$ 1,999,000.00"
City of Ontario,Vine Ave & B St Bike Boulevard Project,San Bernardino,"$ 4,392,000.00","$ 4,881,000.00"


Implemeting Agency Name,Total Applications,Success Rate
City of Ontario,3,33.33%
City of Perris,1,100.00%
Riverside County,9,11.11%
San Bernardino County,2,50.00%
California Department of Transportation,1,0.00%
City of Adelanto,1,0.00%
City of Barstow,1,0.00%
City of Big Bear Lake,2,0.00%
City of Cathedral City,1,0.00%
City of Coachella,1,0.00%


### District 9 - Bishop

In [43]:
_report_utils.map_dist_proj(funded, df, 9)

alt.Chart(...)

Implemeting Agency Name,Project Name,County,Total Atp Funds,Total Project Cost


Implemeting Agency Name,Total Applications,Success Rate
California Department of Transportation,1,0.00%
City of Bishop,1,0.00%
City of Tehachapi,2,0.00%


### District 10 - Stockton

In [44]:
_report_utils.map_dist_proj(funded, df,  10)

alt.Chart(...)

Implemeting Agency Name,Project Name,County,Total Atp Funds,Total Project Cost
Mariposa County,Mariposa Elementary School Connectivity Project,Mariposa,"$ 1,900,000.00","$ 1,900,000.00"
Mariposa County,Mariposa Creek Parkway,Mariposa,"$ 4,415,000.00","$ 5,176,000.00"
Tuolumne County,Jamestown Community Connectivity Project,Tuolumne,"$ 2,071,000.00","$ 2,300,000.00"


Implemeting Agency Name,Total Applications,Success Rate
Mariposa County,2,100.00%
Tuolumne County,1,100.00%
Calaveras County,1,0.00%
California Department of Transportation,1,0.00%
City of Escalon,1,0.00%
City of Lodi,1,0.00%
City of Los Banos,1,0.00%
City of Manteca,2,0.00%
City of Modesto,1,0.00%
City of Patterson,1,0.00%


### District 11 - San Diego

In [45]:
_report_utils.map_dist_proj(funded, df, 11)

alt.Chart(...)

Implemeting Agency Name,Project Name,County,Total Atp Funds,Total Project Cost
City of Imperial Beach,9th St Active Transportation Corridor,San Diego,"$ 3,018,000.00","$ 3,354,000.00"
City of National City,Highland Avenue Inter-City Bike Connection,San Diego,"$ 1,343,000.00","$ 1,897,000.00"
City of Oceanside,Laurel Elementary Safe Routes to School,San Diego,"$ 1,522,000.00","$ 1,535,000.00"
San Diego Association of Governments (SANDAG),Orange Family Friendly Street Project,San Diego,"$ 4,317,000.00","$ 5,660,000.00"


Implemeting Agency Name,Total Applications,Success Rate
City of Imperial Beach,1,100.00%
City of National City,3,33.33%
City of Oceanside,3,33.33%
San Diego Association of Governments (SANDAG),4,25.00%
City o El Cajon,1,0.00%
City of Chula Vista,2,0.00%
City of Coronado,1,0.00%
City of El Cajon,1,0.00%
City of El Centro,1,0.00%
City of Encinitas,2,0.00%


### District 12 - Irvine

In [46]:
_report_utils.map_dist_proj(funded, df, 12)

alt.Chart(...)

Implemeting Agency Name,Project Name,County,Total Atp Funds,Total Project Cost


Implemeting Agency Name,Total Applications,Success Rate
City of Anaheim,1,0.00%
City of Brea,1,0.00%
City of Buena Park,1,0.00%
City of Costa Mesa,1,0.00%
City of Fullerton,1,0.00%
City of Irvine,1,0.00%
City of La Habra,1,0.00%
City of Laguna Hills,1,0.00%
City of Orange,1,0.00%
City of Rancho Santa Margarita,1,0.00%
